資料收集 train test

In [ ]:
# 輸入 filepath
def filepath()

In [ ]:
path = "/Users/emily/Desktop/Research/oversampling_python"
dirs = os.listdir(path+"/data/pima-5-fold")
os.chdir(path+"/data/pima-5-fold")
# 到時候 path 這邊也要 loop 不同資料集的資料夾
# 输出所有文件和文件夹
train = []
test = []
allfile = []
for file in dirs:
    if("xlsx" in file):
        allfile.append(file)
        if("tra" in file ):
            train.append(file)
        elif("tst" in file):
            test.append(file)
allfile

純 RCSMOTE 的預測

In [1]:
# 純 RCSMOTE 的預測
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
import statistics
from sklearn import preprocessing
from sklearn import metrics
os.chdir("/Users/emily/Desktop/Research/oversampling_python/data/pima-5-fold")
accuracies=[]
oversampleRCSMOTE = [] # 全部 5 個 train data 所訓練出來的 oversample data
for i in train:
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    #data = data.iloc[:,1:]

    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    BSnn = getBSnn_Majnn(3,finaldata,output,data)[0]
    Majnn = getBSnn_Majnn(3,finaldata,output,data)[1]
    Range = range_value(3,finaldata,output,data)
    P_max = range_value(3,finaldata,output,data)[0]
    Range = range_value(3,finaldata,output,data)[2]
    alldata = data.T
    classCount = classprocess(output)
    
    tempover = []
    for i in range(len(classCount)):
        if(int(classCount[i][1]) > 0):
            over = Populate(int(classCount[i][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[i][0])
            tempover.extend(over)
            
            length = alldata.shape[1]
            for j in range(len(over)):
                alldata[length+j] = over[j]
    X = alldata.T.iloc[:,:alldata.shape[0]-1]
    y = alldata.T.iloc[:,alldata.shape[0]-1]
    clf = DecisionTreeClassifier()
    clf = clf.fit(X, y) # 訓練 model
    oversampleRCSMOTE.append(tempover)
    
    test_file = pd.read_excel(test[i],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    """
    le = preprocessing.LabelEncoder()
    for i in range(test_data.shape[1]):
        test_data.iloc[:,i] = le.fit_transform(test_data.iloc[:,i]) 
    """
    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
   
    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)


NameError: name 'train' is not defined

In [ ]:
# ranodom half SMOTE 
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []

for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    output = np.array(data.iloc[:,l]);
    classCount = classprocess(output)
    finaldata = np.array(data.iloc[:,:l])
    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    for index,element in enumerate(X_smote):
        temp = np.append(element,[y_smote[index]])
        alloverSMOTE.append(temp)
    overSMOTE.append(alloverSMOTE)
    alloverSMOTE =[]
    
    for i in range(len(classCount)):
        count = math.floor(int(classCount[i][1])/2); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
    
    randomtemp = []

    
    for index in randomIndex:
       
        randomtemp.append(overSMOTE[ii][index])
        
    randomSMOTE.append(randomtemp)
    print(ii," ",len(randomtemp))
    print(np.array(randomSMOTE).shape)
    #print("actual",len(randomSMOTE[ii]))
    
    #print("we",len(randomSMOTE[ii]))
np.array(randomSMOTE).shape

#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
# random half RCSMOTE
randomRCSMOTE = []
for ii,i in enumerate(train):
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    #data = data.iloc[:,1:]

    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    BSnn = getBSnn_Majnn(3,finaldata,output,data)[0]
    Majnn = getBSnn_Majnn(3,finaldata,output,data)[1]
    Range = range_value(3,finaldata,output,data)
    P_max = range_value(3,finaldata,output,data)[0]
    Range = range_value(3,finaldata,output,data)[2]
    alldata = data.T
    classCount = classprocess(output)
    randomIndex = []
    tempover = []
    for j in range(len(classCount)):
        if(int(classCount[j][1]) > 0):
            over = Populate(int(classCount[j][1]),BSnn,Majnn,Range,P_max,finaldata,classCount[j][0])
            tempover.append(over)
            #length = alldata.shape[1]
    oversampleRCSMOTE.append(tempover)
 
    
    for a in range(len(classCount)): # 產生 random index
        count = math.ceil(int(classCount[a][1])/2); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(0,len(oversampleRCSMOTE[ii])-1) for _ in range(count)]) 
    
    randomtemp = []
    #print("randomindex",len(randomIndex))
    
    for index in randomIndex:
        randomtemp.append(oversampleRCSMOTE[ii][index])
    randomRCSMOTE.append(randomtemp)
    #print("actual",len(randomSMOTE[ii]))
 

    


In [ ]:
# random SMOTE + RCSMOTE 各一半
allRandomHalf = []
temp = []
for i in range(len(randomRCSMOTE)):
    temp = randomRCSMOTE[i] + randomSMOTE[i]
    allRandomHalf.append(temp)


In [ ]:
# 跟原始資料合併 + Decision tree
# random SMOTE + RCSMOTE 各一半的結果  
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    output = np.array(data.iloc[:,l]);
    finaldata = np.array(data.iloc[:,:l])
    classCount = classprocess(output)
    data = data.T
    
    
    for i in range(len(allRandomHalf[index])): # 185
        #for j in range(len(allRandomHalf[index][0])): #9
            datalength = data.shape[1]
            data[datalength+i+1] = allRandomHalf[index][j]
    #print(data)  
        #print(datalength)
        #print(allRandomHalf[index][i],"sl;d")
    mergeRandom = data.T
    l = mergeRandom.shape[1]-1
    output = np.array(mergeRandom.iloc[:,l]);
    finaldata = np.array(mergeRandom.iloc[:,:l])
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
   
    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)

#len(mergeRandom[0][0])
